# Fetch weather data for each long and lat data point
### This notebook will iterate over each row of nasa data set and will fetch 24 hours weather data
### Approximately 10k api calls will be made to create the consolidated data

In [79]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import time

In [42]:
nasa = pd.read_csv('data/Wildfire_Data_2020-2023.csv')

In [43]:
nasa.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,37.45576,-121.93266,298.7,0.38,0.58,1/4/20,1042,1,VIIRS,n,2.0NRT,276.9,0.2,N
1,37.31750,-122.09219,295.9,0.37,0.58,1/4/20,1042,1,VIIRS,n,2.0NRT,280.4,0.5,N
2,37.31704,-122.09343,301.3,0.42,0.45,1/5/20,1024,1,VIIRS,n,2.0NRT,280.6,0.9,N
3,37.31878,-122.09222,296.0,0.41,0.37,1/6/20,1006,1,VIIRS,n,2.0NRT,280.0,0.8,N
4,37.45645,-121.93143,305.3,0.41,0.37,1/6/20,1006,1,VIIRS,n,2.0NRT,276.6,0.9,N


In [44]:
nasa.dtypes

latitude      float64
longitude     float64
brightness    float64
scan          float64
track         float64
acq_date       object
acq_time        int64
satellite       int64
instrument     object
confidence     object
version        object
bright_t31    float64
frp           float64
daynight       object
dtype: object

## removing duplicates

In [45]:
nasa[nasa.duplicated(subset=['acq_date', 'acq_time'])]

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
1,37.31750,-122.09219,295.90,0.37,0.58,1/4/20,1042,1,VIIRS,n,2.0NRT,280.40,0.50,N
4,37.45645,-121.93143,305.30,0.41,0.37,1/6/20,1006,1,VIIRS,n,2.0NRT,276.60,0.90,N
6,37.45734,-121.93303,302.80,0.41,0.37,1/7/20,948,1,VIIRS,n,2.0NRT,275.90,0.60,N
7,37.31814,-122.09075,302.40,0.42,0.37,1/7/20,948,1,VIIRS,n,2.0NRT,280.10,1.00,N
8,37.18447,-121.68083,298.70,0.41,0.37,1/7/20,948,1,VIIRS,n,2.0NRT,276.20,0.80,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,37.03717,-121.52206,328.02,0.44,0.38,12/2/23,2053,1,VIIRS,n,2.0NRT,280.33,5.49,D
10048,37.18330,-121.68040,295.06,0.39,0.36,12/6/23,956,1,VIIRS,n,2.0NRT,280.88,0.63,N
10049,37.21344,-121.90060,295.62,0.39,0.36,12/6/23,956,1,VIIRS,n,2.0NRT,280.92,0.24,N
10053,37.18374,-121.67919,298.30,0.39,0.36,12/11/23,1002,1,VIIRS,n,2.0NRT,277.27,0.47,N


In [46]:
nasa.drop_duplicates(subset=['acq_date', 'acq_time'], inplace=True)

In [47]:
nasa.shape

(999, 14)

In [51]:
# Step 1: Convert date columns to consistent format
nasa['acq_date'] = pd.to_datetime(nasa['acq_date'], format='%m/%d/%y')

# Step 2: Convert time column to match weather format
nasa['acq_time'] = nasa['acq_time'].astype(str).str.zfill(4)  # Ensure leading zeros for hours < 10
nasa['acq_time'] = pd.to_datetime(nasa['acq_time'], format='%H%M').dt.time


In [52]:
nasa.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,37.45576,-121.93266,298.7,0.38,0.58,2020-01-04,10:42:00,1,VIIRS,n,2.0NRT,276.9,0.2,N
2,37.31704,-122.09343,301.3,0.42,0.45,2020-01-05,10:24:00,1,VIIRS,n,2.0NRT,280.6,0.9,N
3,37.31878,-122.09222,296.0,0.41,0.37,2020-01-06,10:06:00,1,VIIRS,n,2.0NRT,280.0,0.8,N
5,37.18483,-121.67795,296.7,0.41,0.37,2020-01-07,09:48:00,1,VIIRS,n,2.0NRT,276.9,0.8,N
9,37.31736,-122.08982,302.0,0.39,0.36,2020-01-12,09:54:00,1,VIIRS,n,2.0NRT,274.3,0.5,N


# setting date time formate

In [53]:
nasa['acq_time'] = pd.to_datetime(nasa['acq_time'], format='%H:%M:%S')

# Round to the nearest hour
nasa['round_time'] = nasa['acq_time'].dt.round('H')

# Convert back to time format (HH:MM:SS) 
nasa['round_time'] = nasa['round_time'].dt.time
nasa['acq_time'] = nasa['acq_time'].dt.time

In [57]:
nasa

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,round_time
0,37.45576,-121.93266,298.70,0.38,0.58,2020-01-04,10:42:00,1,VIIRS,n,2.0NRT,276.90,0.20,N,11:00:00
2,37.31704,-122.09343,301.30,0.42,0.45,2020-01-05,10:24:00,1,VIIRS,n,2.0NRT,280.60,0.90,N,10:00:00
3,37.31878,-122.09222,296.00,0.41,0.37,2020-01-06,10:06:00,1,VIIRS,n,2.0NRT,280.00,0.80,N,10:00:00
5,37.18483,-121.67795,296.70,0.41,0.37,2020-01-07,09:48:00,1,VIIRS,n,2.0NRT,276.90,0.80,N,10:00:00
9,37.31736,-122.08982,302.00,0.39,0.36,2020-01-12,09:54:00,1,VIIRS,n,2.0NRT,274.30,0.50,N,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10054,36.97666,-121.54601,330.14,0.38,0.36,2023-12-12,21:03:00,1,VIIRS,n,2.0NRT,291.13,2.34,D,21:00:00
10056,37.09114,-122.15366,327.73,0.53,0.42,2023-12-13,20:46:00,1,VIIRS,n,2.0NRT,287.87,6.63,D,21:00:00
10057,37.45424,-121.93408,299.06,0.54,0.42,2023-12-23,09:38:00,1,VIIRS,n,2.0NRT,278.06,1.04,N,10:00:00
10058,37.45675,-121.93478,305.59,0.32,0.55,2023-12-24,09:17:00,1,VIIRS,n,2.0NRT,276.24,0.71,N,09:00:00


# fetching weather data

In [55]:
import openmeteo_requests

import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [62]:
url = "https://archive-api.open-meteo.com/v1/archive"

# testing the weather api

In [66]:
#37.45576 -121.93266 2020-01-04
params = {
	"latitude": 37.45576,
	"longitude": -121.93266,
	"start_date": "2020-01-04",
	"end_date": "2020-01-04",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
	"timezone": "America/Los_Angeles"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 37.4340934753418°N -121.95478820800781°E
Elevation 2.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -28800 s
                        date  temperature_2m  relative_humidity_2m  \
0  2020-01-04 08:00:00+00:00        7.552500             85.652565   
1  2020-01-04 09:00:00+00:00        6.952500             88.930214   
2  2020-01-04 10:00:00+00:00        7.852500             88.393105   
3  2020-01-04 11:00:00+00:00        7.302500             94.013023   
4  2020-01-04 12:00:00+00:00        7.402500             92.413055   
5  2020-01-04 13:00:00+00:00        7.652500             93.065361   
6  2020-01-04 14:00:00+00:00        9.352500             91.588013   
7  2020-01-04 15:00:00+00:00       10.302500             90.419762   
8  2020-01-04 16:00:00+00:00       10.652500             89.837593   
9  2020-01-04 17:00:00+00:00       12.052500             84.972466   
10 2020-01-04 18:00:00+00:00       13.602500             79.127151   
11 2020-01

# removing time component from date column

In [69]:
nasa['acq_date'] = pd.to_datetime(nasa['acq_date']).dt.date

In [70]:
nasa

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,round_time
0,37.45576,-121.93266,298.70,0.38,0.58,2020-01-04,10:42:00,1,VIIRS,n,2.0NRT,276.90,0.20,N,11:00:00
2,37.31704,-122.09343,301.30,0.42,0.45,2020-01-05,10:24:00,1,VIIRS,n,2.0NRT,280.60,0.90,N,10:00:00
3,37.31878,-122.09222,296.00,0.41,0.37,2020-01-06,10:06:00,1,VIIRS,n,2.0NRT,280.00,0.80,N,10:00:00
5,37.18483,-121.67795,296.70,0.41,0.37,2020-01-07,09:48:00,1,VIIRS,n,2.0NRT,276.90,0.80,N,10:00:00
9,37.31736,-122.08982,302.00,0.39,0.36,2020-01-12,09:54:00,1,VIIRS,n,2.0NRT,274.30,0.50,N,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10054,36.97666,-121.54601,330.14,0.38,0.36,2023-12-12,21:03:00,1,VIIRS,n,2.0NRT,291.13,2.34,D,21:00:00
10056,37.09114,-122.15366,327.73,0.53,0.42,2023-12-13,20:46:00,1,VIIRS,n,2.0NRT,287.87,6.63,D,21:00:00
10057,37.45424,-121.93408,299.06,0.54,0.42,2023-12-23,09:38:00,1,VIIRS,n,2.0NRT,278.06,1.04,N,10:00:00
10058,37.45675,-121.93478,305.59,0.32,0.55,2023-12-24,09:17:00,1,VIIRS,n,2.0NRT,276.24,0.71,N,09:00:00


## empty dataframe to hold the api responses

In [112]:
hourly_dataframe = pd.DataFrame()
print(hourly_dataframe)

Empty DataFrame
Columns: []
Index: []


In [113]:
hourly_dataframe

""


## iterating over the long,lat column and fetching the weather data

In [114]:
for index, row in nasa.iterrows():
    print('This is row--->',index)
    print(row["latitude"], row["longitude"], row['acq_date'])
    
    #37.45576 -121.93266 2020-01-04
    params = {
    	"latitude": row["latitude"],
    	"longitude": row["longitude"],
    	"start_date": row['acq_date'],
    	"end_date": row['acq_date'],
    	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
    	"timezone": "America/Los_Angeles"
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
    hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
    hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
    hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
    hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
    hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
    hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
    hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
    	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = hourly.Interval()),
    	inclusive = "left"
    )}
    hourly_data["latitude"] = row["latitude"]
    hourly_data["longitude"] = row["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
    hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
    hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
    hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
    hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
    hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
    hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm
    hourly_dataframe=pd.concat([hourly_dataframe,pd.DataFrame(data = hourly_data)]) 
    time.sleep(1)
    


This is row---> 0
37.45576 -121.93266 2020-01-04
Coordinates 37.4340934753418°N -121.95478820800781°E
Elevation 2.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -28800 s
This is row---> 2
37.31704 -122.09343 2020-01-05
Coordinates 37.293495178222656°N -121.9893798828125°E
Elevation 207.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -28800 s
This is row---> 3
37.31878 -122.09222 2020-01-06
Coordinates 37.293495178222656°N -121.9893798828125°E
Elevation 192.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -28800 s
This is row---> 5
37.18483 -121.67795 2020-01-07
Coordinates 37.152896881103516°N -121.78572082519531°E
Elevation 163.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -28800 s
This is row---> 9
37.31736 -122.08982 2020-01-12
Coordinates 37.293495178222656°N -121.9893798828125°E
Elevation 179.0 m asl
Timezone b'America/Los_Angeles' b'PST'
Timezone difference to GMT+0 -

### saving the data into csv

In [115]:
hourly_dataframe.to_csv('long_lat_weather_data.csv')

In [124]:
sa_hourly_dataframe = pd.DataFrame()

### fetching the weather data for the san antonio valley

In [125]:
#37.45576 -121.93266 2020-01-04
params = {
    "latitude": 37.3918,
    "longitude": 121.4904,
    "start_date": '2020-01-01',
    "end_date": '2023-12-31',
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
    "timezone": "America/Los_Angeles"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}
hourly_data["latitude"] = row["latitude"]
hourly_data["longitude"] = row["longitude"]
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm
sa_hourly_dataframe=pd.DataFrame(data = hourly_data) 
time.sleep(1)



Coordinates 37.36379623413086°N 121.43426513671875°E
Elevation 15.0 m asl
Timezone b'America/Los_Angeles' b'PDT'
Timezone difference to GMT+0 -25200 s


In [126]:
sa_hourly_dataframe

,date,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,...,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,2020-01-01 07:00:00+00:00,37.45598,-121.93182,4.3185,20.565947,-16.3815,0.146064,0.0,0.0,0.0,...,327.528839,15.840000,-0.6315,0.9685,6.7685,15.968499,0.246,0.228,0.066,0.180
1,2020-01-01 08:00:00+00:00,37.45598,-121.93182,1.2185,32.135635,-13.6315,-3.140542,0.0,0.0,0.0,...,41.633450,10.440001,-0.5815,0.9685,6.7685,15.968499,0.246,0.228,0.067,0.180
2,2020-01-01 09:00:00+00:00,37.45598,-121.93182,-1.2815,62.970901,-7.4315,-5.792558,0.0,0.0,0.0,...,57.144287,18.719999,-0.5815,0.9685,6.7185,15.968499,0.245,0.228,0.067,0.180
3,2020-01-01 10:00:00+00:00,37.45598,-121.93182,-4.0315,78.410561,-7.2315,-8.409879,0.0,0.0,0.0,...,64.798920,18.359999,-0.6315,0.9685,6.7185,15.968499,0.245,0.228,0.067,0.180
4,2020-01-01 11:00:00+00:00,37.45598,-121.93182,-3.5815,72.100960,-7.8815,-7.304724,0.0,0.0,0.0,...,107.525658,11.879999,-0.6815,0.9685,6.7185,15.968499,0.245,0.228,0.067,0.180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2024-01-01 02:00:00+00:00,37.45598,-121.93182,0.5185,64.867241,-5.3315,-3.530695,0.0,0.0,0.0,...,311.185822,20.519999,-0.3315,-0.2315,3.3685,14.268499,0.309,0.253,0.155,0.232
35060,2024-01-01 03:00:00+00:00,37.45598,-121.93182,1.5685,60.150349,-5.3315,-1.844369,0.0,0.0,0.0,...,315.000092,20.519999,-0.2315,-0.2315,3.3685,14.218499,0.309,0.253,0.155,0.232
35061,2024-01-01 04:00:00+00:00,37.45598,-121.93182,2.4685,55.772015,-5.4815,-0.471454,0.0,0.0,0.0,...,270.000000,15.480000,-0.0815,-0.2315,3.3685,14.218499,0.309,0.253,0.155,0.232
35062,2024-01-01 05:00:00+00:00,37.45598,-121.93182,3.0685,52.643452,-5.6815,0.046270,0.0,0.0,0.0,...,216.869980,11.879999,0.0685,-0.2315,3.3685,14.218499,0.308,0.253,0.155,0.232


In [127]:
sa_hourly_dataframe.to_csv('san_atonio_weather_data.csv')